In [1]:
import langchain
import openai
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chromadb
from  PyPDF2 import PdfReader
import os
from sentence_transformers import SentenceTransformer
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import (
    ConversationalRetrievalChain,
    LLMChain
)
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores import Chroma
import keyboard
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings


In [2]:
os.environ['OPENAI_API_KEY']=''


In [3]:

def get_vector_store(folder_path):
    
    openai_key=os.environ['OPENAI_API_KEY']
    all_text=[]

    for i in os.listdir(folder_path):
        filename=os.path.join(str(folder_path),i)
        if filename.endswith('.pdf'):
            reader = PdfReader(filename)
            page_texts = [page.extract_text() for page in reader.pages]
            text = " ".join(page_texts)
            all_text.append(text)

        elif filename.endswith('.txt'):
            with open(filename, encoding="utf8") as f:
                all_text.append(f.read())


    documentation_text="/n".join(all_text)
    text_splitter =  CharacterTextSplitter(
        separator='\n',
        chunk_overlap=0,
        chunk_size=600
    )

    texts = text_splitter.create_documents([documentation_text])
    texts=[doc.page_content for doc in texts]

    embedding = OpenAIEmbeddings()

    vector_store=Chroma.from_texts(
      texts=texts,
      embedding=embedding,
      persist_directory=r'C:\Users\Naveen Reddy\Downloads\chatbot', 
        )


In [4]:

def run_chat_bot():
    
    chat_history=[]
    
    embedding = OpenAIEmbeddings()

    
    vectordb = Chroma(persist_directory=r'C:\Users\Naveen Reddy\Downloads\chatbot', embedding_function=embedding)

    template = """Given the following chat history and a follow up question, rephrase the follow up input question to be a standalone question.
    Or end the conversation if it seems like it's done.
    Chat History:\"""
    {chat_history}
    \"""
    Follow Up Input: \"""
    {question}
    \"""
    Standalone question:"""

    condense_question_prompt = PromptTemplate.from_template(template)


    # [Prompt]
    prompt_template = """You are a Bot assistant answering any questions about support queries for hyperverge company.
    You are given a question and a set of information and carefully observe the information provided because the clients ask some tricky questions.
    If the user's question requires you to provide specific information from the information, give your answer based only on the examples provided below. DON'T generate an answer that is NOT written in the provided examples.
    If you don't find the answer to the user's question with the examples provided to you below, answer that you didn't find the answer in the documentation and propose him to rephrase his query with more details.
    Use bullet points if you have to make a list, only if necessary.

    QUESTION: {question}

    DOCUMENTS:
    =========
    {context}
    =========
    Finish by proposing your help for anything else.
    """


    qa_prompt= PromptTemplate.from_template(prompt_template)

    llm = OpenAI(temperature=0)

    streaming_llm = OpenAI(
        streaming=True,
        callback_manager=CallbackManager([
            StreamingStdOutCallbackHandler()
        ]),
        verbose=True,
        max_tokens=250,
        temperature=0
    )

    question_generator = LLMChain(
    llm=llm,
    prompt=condense_question_prompt
    )

    doc_chain = load_qa_chain(
    llm=streaming_llm,
    chain_type="stuff",
    prompt=qa_prompt
    )

    chatbot = ConversationalRetrievalChain(
    retriever=vectordb.as_retriever(),
    combine_docs_chain=doc_chain,
    question_generator=question_generator,
    )


    question = input("Hi! What are you looking for today?")
    chat_history
    while True:
        result = chatbot(
            {"question": question, "chat_history": chat_history}
        )
        print("\n")



        chat_history.append((result["question"], result["answer"]))
        if keyboard.is_pressed('q'):  
            break

        question = input()
        if question.lower() in ['exit', 'quit']:
            print("Thank you for contacting support...")
            break



In [5]:
folder_path=r'C:\Users\Naveen Reddy\Downloads\chatbot\Documentation\Documentation'
get_vector_store(folder_path)

Using embedded DuckDB with persistence: data will be stored in: C:\Users\Naveen Reddy\Downloads\chatbot


In [ ]:
run_chat_bot()

Using embedded DuckDB with persistence: data will be stored in: C:\Users\Naveen Reddy\Downloads\chatbot


Hi! What are you looking for today?hi

Sorry, I didn't understand your question. Could you please rephrase it with more details? If you need help with anything else, please let me know.

what is hyperverge api

Answer: The Hyperverge API is a REST-based API that uses standard HTTP verbs and status codes. It uses JSON format for requests and responses and requires authentication with an AppID and AppKey. It provides access to HyperVerge and your team for debugging various issues that might have occurred during AI processing. It also includes a Liveness API, Score Based Name Match API, and Field Match API. The Score Based Name Match API can be used to compare a name from one source with the same name from another source, and we consider initials in a name to be 2 or lesser characters.

Is there anything else I can help you with?

what liveness api

Answer: The Liveness API is a proprietary API from HyperVerge that is used to ascertain if an image of a user is not a photo of a photo (not 